In [16]:
#FINAL FINAL FINAL

import cvxpy as cp
import numpy as np

num_students = 1000
num_universities = 20

np.random.seed(42)  # For reproducibility
student_data = np.column_stack((
    np.round(np.random.uniform(2.0, 4.0, num_students), 2),  
    np.random.randint(1000, 1601, num_students),            
    np.random.randint(1, 101, num_students)               
))

university_requirements = np.column_stack((
    np.round(np.random.uniform(2.5, 4.0, num_universities), 2),  
    np.random.randint(1000, 1601, num_universities),             
    np.random.randint(50, 201, num_universities)                
))

# Create students' preference scores
preference_scores = np.array([np.random.choice(range(1, 21), 
    num_universities, replace=False) for _ in range(num_students)])

# University's capacity size for incoming students
university_capacity_size = np.random.randint(800, 1201, size=20)

x = cp.Variable((num_students, num_universities), boolean=True)

# Sort students
sorted_indices = np.lexsort((student_data[:, 2], student_data[:, 1], -student_data[:, 0]))
sorted_student_data = student_data[sorted_indices]
sorted_preference_scores = preference_scores[sorted_indices]

objective = cp.Maximize(cp.sum(cp.multiply(sorted_preference_scores, x)))

constraints = []
#Eligibility constraint
eligibility_matrix = np.zeros((num_students, num_universities), dtype=int)
for i in range(num_students):
    for j in range(num_universities):
        gpa, sat, rank = sorted_student_data[i]
        min_gpa, min_sat, max_rank = university_requirements[j]
        if gpa >= min_gpa and sat >= min_sat and rank <= max_rank:
            eligibility_matrix[i, j] = 1
        constraints.append(x[i, j] <= eligibility_matrix[i, j])  # Ensure eligibility 

# Capacity constraint for universities
for j in range(num_universities):
    constraints.append(cp.sum(x[:, j]) <= university_capacity_size)  # University capacity

problem = cp.Problem(objective, constraints)
problem.solve(verbose=False)

print("Total preference score:", objective.value)
# Print the results with a star (*) for students admitted to their top university preference
print("Matching results:")
for i, original_index in enumerate(sorted_indices):
    admitted_universities = []
    top_preference_universities = [j + 1 for j in range(num_universities) 
        if sorted_preference_scores[i, j] == 20]  # 1-indexed
    
    for j in range(num_universities):
        if x.value is not None and x.value[i, j] > 0.5:
            admitted_universities.append(j + 1)
    
    if admitted_universities:
        stars = ["*" if uni in top_preference_universities else "" for uni in admitted_universities]
        results = ", ".join(f"{uni}{star}" for uni, star in zip(admitted_universities, stars))
        print(f"Student {original_index + 1} admitted to University/Universities: {results}")
    else:
        print(f"Student {original_index + 1} not admitted to any university")


Total preference score: 30532.0
Matching results:
Student 532 admitted to University/Universities: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16*, 17, 18, 19, 20
Student 476 admitted to University/Universities: 1, 6, 8, 10, 12, 13, 14, 15, 16, 19, 20
Student 533 admitted to University/Universities: 6, 7, 8*, 13, 14, 15, 16, 17, 19, 20
Student 848 admitted to University/Universities: 1, 3, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17, 19, 20
Student 262 not admitted to any university
Student 391 admitted to University/Universities: 6, 7, 8, 13, 14, 15, 16, 17, 20
Student 706 admitted to University/Universities: 1, 3, 6, 7, 8*, 10, 12, 13, 14, 15, 16, 17, 20
Student 500 admitted to University/Universities: 8, 10
Student 811 admitted to University/Universities: 6, 8
Student 442 admitted to University/Universities: 1, 6, 8, 10, 14, 16, 20
Student 933 admitted to University/Universities: 1, 6, 8, 10, 12, 13, 14, 15, 16, 20
Student 447 admitted to University/Universities: 1, 6, 8, 12, 13, 14*

In [11]:
np.set_printoptions(threshold=np.inf, linewidth=200)
print(university_capacity_size)

[1146  860  965  959  950  910  836  802  812  961 1143 1131 1063  882 1064 1133 1179 1018  945 1097]
